# YOLOv8n-Pose 실행 및 Keypoints 추출

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import json
import torch


In [2]:
# GPU가 사용 가능한지 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"💻 Using device: {device}")

💻 Using device: cuda


In [3]:
# YOLOv8n-Pose 모델 불러오기 (GPU 사용)
model = YOLO("yolov8n-pose.pt").to(device)

In [6]:
# 이미지 데이터 경로 설정
image_folder = r"C:\Users\admin\Desktop\ZB\ZB_DL_proj\image_dataset"
output_folder = r"C:\Users\admin\Desktop\ZB\ZB_DL_proj\keypoints_json"

os.makedirs(output_folder, exist_ok=True)

In [7]:
# 이미지 처리 및 Keypoints 좌표 저장
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)

    # YOLOv8n-Pose 실행
    results = model(image, device=device)

    for result in results:
        if result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()  # (x, y) 좌표만 추출
            keypoints_list = keypoints.tolist()

            # JSON 파일 저장
            output_path = os.path.join(output_folder, f"{image_name.split('.')[0]}.json")
            with open(output_path, "w") as f:
                json.dump({"image": image_name, "keypoints": keypoints_list}, f)

print("✅ Keypoints 데이터 추출 완료!")


0: 384x640 1 person, 36.2ms
Speed: 19.0ms preprocess, 36.2ms inference, 219.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 21.7ms
Speed: 7.1ms preprocess, 21.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 19.4ms
Speed: 2.0ms preprocess, 19.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 19.2ms
Speed: 2.0ms preprocess, 19.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 6.4ms postprocess per image at shape (1,

# Keypoints 데이터 벡터 변환 및 자동 레이블링

In [8]:
keypoints_folder = r"C:\Users\admin\Desktop\ZB\ZB_DL_proj\keypoints_json"

all_vectors = []
labels = []

In [9]:
# YOLOv8n-Pose의 기본 Keypoints 개수 (17개)
NUM_KEYPOINTS = 17
VECTOR_SIZE = NUM_KEYPOINTS * 2  # (x, y) 좌표만 사용 → 34차원 벡터

In [10]:
for json_file in os.listdir(keypoints_folder):
    file_path = os.path.join(keypoints_folder, json_file)

    with open(file_path, "r") as f:
        data = json.load(f)
        keypoints = np.array(data["keypoints"])[:, :2]  # (x, y) 좌표만 추출

        # Keypoints가 비어있는 경우 (탐지 실패) → 0으로 초기화
        if keypoints.size == 0:
            keypoints = np.zeros((NUM_KEYPOINTS, 2))  # 17개의 (0,0) 좌표로 채움
        else:
            keypoints = keypoints.reshape(-1, 2)  # 강제로 (n, 2) 형태로 변환

        # Keypoints 개수 확인 (누락된 경우 0으로 패딩)
        if keypoints.shape[0] < NUM_KEYPOINTS:
            missing_keypoints = NUM_KEYPOINTS - keypoints.shape[0]
            padding = np.zeros((missing_keypoints, 2))  # (x,y) 좌표 패딩
            keypoints = np.vstack([keypoints, padding])  # 패딩 추가

        # 벡터 변환 (Flatten)
        keypoint_vector = keypoints.flatten()

        # 벡터 크기가 일관되지 않을 경우 예외 처리
        if keypoint_vector.shape[0] != VECTOR_SIZE:
            keypoint_vector = np.zeros(VECTOR_SIZE)  # 크기가 다르면 0으로 채움

        # **파일명 기반 자동 레이블링**
        label = 1 if "fall" in json_file.lower() else 0  # 'fall' 포함되면 1, 없으면 0

        all_vectors.append(keypoint_vector)
        labels.append(label)

# numpy 배열로 변환하여 저장
X = np.array(all_vectors)
y = np.array(labels)

print(f"✅ 벡터 변환 완료! 데이터 크기: {X.shape}")

✅ 벡터 변환 완료! 데이터 크기: (1000, 34)


In [11]:
# 데이터 저장
np.save("X_keypoints.npy", X)
np.save("y_labels.npy", y)
print("✅ 벡터 데이터 저장 완료! (X_keypoints.npy, y_labels.npy)")

✅ 벡터 데이터 저장 완료! (X_keypoints.npy, y_labels.npy)
